In [51]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')

melb_df = melb_data.copy()

total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area

countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})
dens=(countries_df['population']*1000000)/countries_df['square']
result=dens.mean()

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
years_sold = melb_df['Date'].dt.year
#print(years_sold)
#print('Min year sold:', years_sold.min())
#print('Max year sold:', years_sold.max())
#print('Mode year sold:', years_sold.mode()[0])

melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
#display(melb_df['AgeBuilding'])
melb_df = melb_df.drop('YearBuilt', axis=1)

melb_df['Weekdaysale']=melb_df['Date'].dt.dayofweek
weekendcount=0
for i in melb_df['Weekdaysale']:
    if i==5 or i==6:
        weekendcount+=1
        
#data=pd.read_csv('https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/ufo.csv')
#data['Time']=pd.to_datetime(data['Time'])

#data_year=data['Time'].dt.year
#data_year.mode()

#data['Date']=data['Time'].dt.date
#nv_data=data[data['State']=='NV']
#d=nv_data['Date'].diff()
#d.mean()

def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

street_types = melb_df['Address'].apply(get_street_type)
popular_stypes =street_types.value_counts().nlargest(10).index

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
melb_df = melb_df.drop('Address', axis=1)

def get_weekend(weekday):
    
    if weekday==5 or weekday==6:
        return 1
    return 0
melb_df['Weekend']= melb_df['Weekdaysale'].apply(get_weekend)
melb_df[melb_df['Weekend']==1]['Price'].mean()

sellers=melb_df['SellerG']
popular_sellers =sellers.value_counts().nlargest(49).index
melb_df['SellerG'] = sellers.apply(lambda x: x if x in popular_sellers else 'other')

nels_min=melb_df[melb_df['SellerG']=='Nelson']['Price'].min()
ot_min=melb_df[melb_df['SellerG']=='other']['Price'].min()
print(nels_min/ot_min)

1.297709923664122


In [54]:
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран display(unique_counts)

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца


subs=melb_df['Suburb']
popular_subs =subs.value_counts().nlargest(119).index
melb_df['Suburb'] = subs.apply(lambda x: x if x in popular_subs else 'other')

melb_df['Suburb'] = melb_df['Suburb'].astype('category')

display(melb_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   index            13580 non-null  int64         
 1   Suburb           13580 non-null  category      
 2   Rooms            13580 non-null  int64         
 3   Type             13580 non-null  category      
 4   Price            13580 non-null  float64       
 5   Method           13580 non-null  category      
 6   SellerG          13580 non-null  category      
 7   Date             13580 non-null  datetime64[ns]
 8   Distance         13580 non-null  float64       
 9   Postcode         13580 non-null  int64         
 10  Bedroom          13580 non-null  int64         
 11  Bathroom         13580 non-null  int64         
 12  Car              13580 non-null  int64         
 13  Landsize         13580 non-null  float64       
 14  BuildingArea     13580 non-null  float

None

In [85]:
bike_data= pd.read_csv('data/citibike-tripdata.csv', sep=',')
bike_data['start station id'].count()
subs_data=bike_data[bike_data['usertype']=='Subscriber']
bike_data.shape
subs_data.shape
wmen_data=bike_data[bike_data['gender']==2]
wmen_data.shape
bike_data['end station name'].mode()
bike_data= bike_data.drop('end station id', axis=1)
bike_data= bike_data.drop('start station id', axis=1)
bike_data['age']=2018-bike_data['birth year']
older_than_60=bike_data[bike_data['age']>60]
bike_data['stoptime']=pd.to_datetime(bike_data['stoptime'])
bike_data['starttime']=pd.to_datetime(bike_data['starttime'])
bike_data['trip duration']=bike_data['stoptime']-bike_data['starttime']

def get_weekend(weekday):
    
    if weekday==5 or weekday==6:
        return 1
    return 0
bike_data['Daynum']=bike_data['starttime'].dt.dayofweek
bike_data['Weekend']= bike_data['Daynum'].apply(get_weekend)
weekend_data=bike_data[bike_data['Weekend']==1]
weekend_data.shape

def what_time(hour):
    if 0<=hour<=6:
        return 'night'
    elif 6<hour<=12:
        return 'morning'
    elif 12<hour<=18:
        return 'day'
    elif 18<hour<=23:
        return 'evening'
    
hour_data=bike_data['starttime'].dt.hour
bike_data['time_of_day']=hour_data.apply(what_time)
ntime=bike_data[bike_data['time_of_day']=='night']
ntime.shape


(15085, 17)